In [2]:
import pandas as pd
import requests
import json
import networkx as nx
import matplotlib.pyplot as plt
import itertools

In [3]:
csv = pd.read_csv("MediaCloud/biomass-all-story-urls-20210422214554.csv")

csv.tail(1)


,stories_id,publish_date,title,url,language,ap_syndicated,themes,media_id,media_name,media_url
7177,1908931813,2021-04-21 06:45:00,Why the federal government is buying into the ...,https://grist.org/energy/can-the-federal-gover...,en,False,"environment, energy and power, air pollution, ...",27343,Grist,http://grist.org/


In [1]:
# Save JSON files

ids = csv['stories_id'].tolist()
len(ids)

for index, id in enumerate(ids):

    id = str(id)
    filename = 'json/' + id +'.json'

    try:
        f = open(filename)
        f.close()

    except:
        
        if (index % 10 == 0): print(index, id)
        r = requests.get('https://api.mediacloud.org/api/v2/stories_public/single/' + id + '?key=b2ef1a99a8fdbb84afafe742fd437c0942703072bb98242ba0eed9b8411e1735')
        open(filename, 'wb').write(r.content)



NameError: name 'csv' is not defined

In [3]:



# Working ID 333682533

G = nx.Graph()
labels = {}

id = str(1908931813)

url = 'https://api.mediacloud.org/api/v2/stories_public/single/' + id + '?key=b2ef1a99a8fdbb84afafe742fd437c0942703072bb98242ba0eed9b8411e1735'
r = requests.get(url, allow_redirects=True)

y = json.loads(r.content)
# open(id +'.json', 'wb').write(r.content)

orgs = []

for t in y[0]['story_tags']:
    if (t['tag_set'] == 'cliff_organizations'):
        
        id = t['tags_id']
        name = t['tag']
        
        orgs.append(id)
        
        G.add_node(id)
        labels[id] = name

edges = list(itertools.combinations(orgs, 2))

for e in edges:
    G.add_edge(e[0], e[1], weight=1)


print("Nodes: ", G.nodes()) # returns a list
print("Edges: ", G.edges()) # returns a list

G[9363209]







Nodes:  [9363209, 9366963, 9376410, 9398493, 9584691, 9600394, 10271408]
Edges:  [(9363209, 9366963), (9363209, 9376410), (9363209, 9398493), (9363209, 9584691), (9363209, 9600394), (9363209, 10271408), (9366963, 9376410), (9366963, 9398493), (9366963, 9584691), (9366963, 9600394), (9366963, 10271408), (9376410, 9398493), (9376410, 9584691), (9376410, 9600394), (9376410, 10271408), (9398493, 9584691), (9398493, 9600394), (9398493, 10271408), (9584691, 9600394), (9584691, 10271408), (9600394, 10271408)]


AtlasView({9366963: {'weight': 1}, 9376410: {'weight': 1}, 9398493: {'weight': 1}, 9584691: {'weight': 1}, 9600394: {'weight': 1}, 10271408: {'weight': 1}})